In [7]:
import random
from collections import deque
import math

tabId = []
tabProfit = []
tabPoids = []


In [27]:
#Remplissage des tableaux qui représente le graphe
def lireFichier(f):
    fichier = open(f, "r")
    fichier.readline()
    fichier.readline()
    fichier.readline()
    nbItem = ((fichier.readline()).split(" "))[1]
    maxCap = ((fichier.readline()).split(" "))[1]
    fichier.readline()
    fichier.readline()
    for i in range(int(nbItem)):
        element = (fichier.readline()).split(" ")
        tabId.append(element[0])
        tabProfit.append(element[1])
        tabPoids.append(element[2].replace("\n", ""))

    fichier.close()
    return int(nbItem), int(maxCap)

nbItem, maxCap = lireFichier("data/pi-15-1000-1000-001.kna")


In [41]:
def tabuSearch():
    maxIter = 300
    xmin = solutionInitial()
    x = xmin.copy()
    fmin = sommeProfit(xmin)
    i = 0
    queueTabu = deque([])
    lenTabu = 30
    for i in range (0,maxIter):
        listVoisin, listChangement = listerVoisin(queueTabu, x)
        xbis, bitChanged = maximiserProfit(listVoisin, listChangement)
        deltaf = sommeProfit(xbis) - sommeProfit(x)
        if deltaf <= 0 :
            if(len(queueTabu)==lenTabu):
                queueTabu.popleft()
            queueTabu.append(bitChanged)
        print(queueTabu)
        if sommeProfit(xbis) > sommeProfit(xmin):
            xmin = xbis.copy()
            fmin = sommeProfit(xbis)
        x = xbis.copy()
    return xmin, fmin

def sommeProfit(listItem):
    sum = 0
    i=0
    for i in range(0, nbItem):
        if listItem[i] == 1 :
            sum += int(tabProfit[i])
    return sum

def isAccepted(listItem):
    sum = 0
    i=0
    for i in range(0, nbItem):
        if listItem[i] == 1 :
            sum += int(tabPoids[i])
    return (sum <= maxCap)

def listerVoisin(tabTabu, listItem):
    listVoisin = []
    listChangement = []
    for i in range(0,nbItem):
        voisin = listItem.copy()
        if voisin[i] == 0 :
            voisin[i] = 1
        else :
            voisin[i] = 0
        if not i in tabTabu and isAccepted(voisin):
            listVoisin.append(voisin)
            listChangement.append(i)
    return listVoisin, listChangement

def maximiserProfit(listVoisin, listChangement):
    max=-1
    betterVoisin = []
    bitChanged = -1
    for i in range(0, len(listVoisin)):
        if max < sommeProfit(listVoisin[i]) : 
            max = sommeProfit(listVoisin[i])
            betterVoisin = listVoisin[i]
            bitChanged = listChangement[i]
    print(max, bitChanged)
    return betterVoisin, bitChanged

def solutionInitial():
    return [0] * int(nbItem)

In [42]:
#%%timeit
#zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz
print(tabuSearch())

999 130
deque([])
1998 230
deque([])
2997 415
deque([])
3996 764
deque([])
4824 456
deque([])
3996 456
deque([456])
4821 157
deque([456])
4824 335
deque([456])
4821 335
deque([456, 335])
4824 641
deque([456, 335])
4821 641
deque([456, 335, 641])
4824 833
deque([456, 335, 641])
4821 833
deque([456, 335, 641, 833])
3996 157
deque([456, 335, 641, 833, 157])
4821 257
deque([456, 335, 641, 833, 157])
3996 257
deque([456, 335, 641, 833, 157, 257])
4821 339
deque([456, 335, 641, 833, 157, 257])
3996 339
deque([456, 335, 641, 833, 157, 257, 339])
4821 655
deque([456, 335, 641, 833, 157, 257, 339])
3996 655
deque([456, 335, 641, 833, 157, 257, 339, 655])
4818 381
deque([456, 335, 641, 833, 157, 257, 339, 655])
4824 411
deque([456, 335, 641, 833, 157, 257, 339, 655])
4818 411
deque([456, 335, 641, 833, 157, 257, 339, 655, 411])
3996 381
deque([456, 335, 641, 833, 157, 257, 339, 655, 411, 381])
4818 544
deque([456, 335, 641, 833, 157, 257, 339, 655, 411, 381])
4827 114
deque([456, 335, 641, 833, 

In [ ]:
def recuitSimule():
    T = 100  # Température initiale
    alpha = 0.99  # Facteur de refroidissement
    Tmin = 0.1  # Température minimale
    iterMax = 1000  # Nombre d'itérations

    x = solutionInitial()  # Solution initiale
    x_best = x.copy()
    f_best = sommeProfit(x)
    
    print("Début du recuit simulé...")
    print(f"Température initiale: {T}")
    
    for iter in range(iterMax):
        if T < Tmin:
            break  # on arrete sii la température est trop basse
        
        voisin = genererVoisin(x)
        deltaE = sommeProfit(voisin) - sommeProfit(x)
        
        if deltaE > 0 or random.random() < math.exp(deltaE / T):
            x = voisin.copy()
            if sommeProfit(x) > f_best:
                x_best = x.copy()
                f_best = sommeProfit(x)
        
        T *= alpha  # Réduction de la température
        
        if iter % 100 == 0:  # pour afficher l'état toutes les 100 itérations
            print(f"Iteration {iter}, Température: {T:.2f}, Profit actuel: {sommeProfit(x)}")
    
    print("Recuit simulé terminé!!!!!!")
    return x_best, f_best

def genererVoisin(solution):
    voisin = solution.copy()
    i = random.randint(0, nbItem - 1)  # pour qu'onn change un élément aléatoirement
    voisin[i] = 1 - voisin[i]  
    return voisin if isAccepted(voisin) else solution  

def solutionInitial():
    return [0] * nbItem  

In [ ]:
# test
fichier_test = "data/pi-12-100-1000-001.kna"
lireFichier(fichier_test)
print(f"Capacité max: {maxCap}")
best_solution, best_profit = recuitSimule()
print(f"Meilleure solution trouvée: {best_solution}")
print(f"Profit maximal obtenu: {best_profit}")

Capacité max: 970
Début du recuit simulé...
Température initiale: 100
Iteration 0, Température: 99.00, Profit actuel: 468
Iteration 100, Température: 36.24, Profit actuel: 773
Iteration 200, Température: 13.26, Profit actuel: 773
Iteration 300, Température: 4.86, Profit actuel: 773
Iteration 400, Température: 1.78, Profit actuel: 773
Iteration 500, Température: 0.65, Profit actuel: 773
Iteration 600, Température: 0.24, Profit actuel: 773
Recuit simulé terminé!!!!!!
Meilleure solution trouvée: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Profit maximal obtenu: 773


In [ ]:
def gentique():
    intial = genPolulationInitial()
    
    return 0

def genPolulationInitial():
    sol = [0] * int(nbItem)
    return [sol] * 100

def reproduction():
    #prend les 100 meilleurs au hasard 
    return 0

def croisement():
    #Faire le croisement entre deux solutions au hasard
    return 0

def mutation():
    #Faire muter un bit au hasard
    return 0